In [ ]:
from src import api
from src import wiki
url = "https://lolpros.gg/player/goksi"
player_data = api.get_player_data_by_lolpros(url, last_n=20)

In [ ]:
from constants import *
get_champ_name_to_id()

In [1]:
import psycopg2
from psycopg2 import extras
import pandas as pd
from datetime import datetime
from src import api
from src import wiki

try:
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host="postgresql.r5.websupport.sk",
        database="brch_db",
        user="brch_admin",
        password="Ps8zV>95Dg",
        connect_timeout=600
    )

    with conn.cursor() as cur:
        # Fetch teams data from the database
        cur.execute("SELECT id, name, queryname, url, stats FROM team")
        teams = cur.fetchall()
        teams = pd.DataFrame(teams, columns=["id", "name", "queryname", "url", "stats"])

        for _, team in teams.iterrows():
            # Handle team params
            # ----------------------------
            query = team['queryname']
            team_id = team['id']
            team_picks, bans, bans_against = wiki.get_team_data_from_lol_wiki(query)
            dt_string = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            stats = {
                "picks": team_picks,
                "bans": bans,
                "bans_against": bans_against
            }
            stats = extras.Json(stats)

            # Update team stats
            # ----------------------------
            update_query = """
                UPDATE team
                SET stats = %s, last_update = %s
                WHERE id = %s
            """
            cur.execute(update_query, (stats, dt_string, team_id))


            cur.execute(f"SELECT id, name, lolprosurl, alternativeids FROM player WHERE teamid = {team_id}")
            team_players = cur.fetchall()
            team_players = pd.DataFrame(team_players, columns=["id", "name", "lolprosurl", "alternativeids"])
            for _, player in team_players.iterrows():
                url = player['lolprosurl'] #"https://lolpros.gg/player/goksi"
                player_id = player['id']
                player_data = api.get_player_data_by_lolpros(url, last_n=20)
                player_data = extras.Json(player_data)
                update_player_query = """
                UPDATE player
                SET accounts = %s, last_update = %s
                WHERE id = %s
                """
                cur.execute(update_player_query, (player_data, dt_string, player_id))

            # Commit the changes to the database
            conn.commit()
    
except Exception as e:
    print(e)
finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()


2023-06-12 15:18:40,531 - INFO - Looking for team - stopwatch
2023-06-12 15:18:40,532 - INFO - Query url - https://lol.fandom.com/wiki/Hitpoint_2nd_Division_Challengers/2023_Season/Summer_Season/Match_History
2023-06-12 15:18:40,715 - INFO - stopwatch parsing successful
2023-06-12 15:18:41,923 - INFO - player - shin | ids found - ['wqecxzsa', 'asdfgjhzvxc']
2023-06-12 15:18:45,303 - INFO - wqecxzsa - found 22 games
2023-06-12 15:18:45,308 - INFO - wqecxzsa | soloq : MASTER I 578 | flexq : Unranked
2023-06-12 15:18:47,713 - INFO - asdfgjhzvxc - found 0 games
2023-06-12 15:18:47,717 - INFO - asdfgjhzvxc | soloq : PLATINUM I 75 | flexq : Unranked
2023-06-12 15:18:48,534 - INFO - player - fogzy | ids found - ['ff in head', 'ANDRIY BAMPER']
2023-06-12 15:18:56,223 - INFO - ff in head - found 54 games
2023-06-12 15:18:56,239 - INFO - ff in head | soloq : MASTER I 690 | flexq : Unranked
2023-06-12 15:18:57,107 - INFO - ANDRIY BAMPER - found 0 games
2023-06-12 15:18:57,115 - INFO - ANDRIY BAMP